<a href="https://colab.research.google.com/github/NatashaKhotkina/Amino_acid_transfer_learning_for_antibody_binding_prediction/blob/main/cov2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Reading tha Data

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
train_ly16 = pd.read_csv("/content/gdrive/MyDrive/BI_project/LY16_train_data.csv")
test_ly16 = pd.read_csv("/content/gdrive/MyDrive/BI_project/LY16_test_data.csv")

In [4]:
test_ly16.head()

,Unnamed: 0,junction_aa,v_call,consensus_count,j_call,clonal_frequency,Label,Distance,Antibody
0,56910,KNTGFNCYFPLLAYGFHPTSGVDY,IGHV_RBD,1,IGHJ_RBD,0.000063,0,6,LY16
1,4131,KNPGFNCYHPIVGYGFQPTIGHDF,IGHV_RBD,1,IGHJ_RBD,0.000047,1,9,LY16
2,46782,TTAGFNCYMPITRYGFWPTDGRSW,IGHV_RBD,1,IGHJ_RBD,0.000028,0,12,LY16
3,11781,RNRGFNCYDPIHKYGFHRTNGLNY,IGHV_RBD,1,IGHJ_RBD,0.000047,1,10,LY16
4,33282,KNKKFNCYVPLVTYGFHPTNGVNY,IGHV_RBD,1,IGHJ_RBD,0.000028,0,7,LY16


In [5]:
seq_df = train_ly16['junction_aa'].str.split('', expand = True).drop(columns=[0, 25])

In [6]:
aa_alph = np.unique(seq_df.values).tolist()

In [7]:
len(aa_alph)

20

In [8]:
enc = OneHotEncoder()
encoding = enc.fit_transform(np.array(aa_alph).reshape(-1, 1)).toarray()

In [9]:
encoding_dict = dict(zip(aa_alph, encoding))

In [10]:
def encoding_func(dataset):
  encoded_list = []
  for i in range(len(dataset)):
    seq = dataset['junction_aa'].iloc[i]
    seq_encodded = []
    for letter in seq:
      seq_encodded.append(encoding_dict[letter])
    seq_encodded = np.array(seq_encodded)
    encoded_list.append(seq_encodded)
  encoded_list = np.array(encoded_list)
  return encoded_list


In [11]:
train_ly16_encoded = encoding_func(train_ly16)
test_ly16_encoded = encoding_func(test_ly16)

In [12]:
X_train_ly16_rf = train_ly16_encoded.reshape(train_ly16_encoded.shape[0], (train_ly16_encoded.shape[1]*train_ly16_encoded.shape[2]))
X_test_ly16_rf = test_ly16_encoded.reshape(test_ly16_encoded.shape[0], (test_ly16_encoded.shape[1]*test_ly16_encoded.shape[2]))

In [13]:
y_train_ly16 = train_ly16['Label']
y_test_ly16 = test_ly16['Label']

# Random forest

## Ly16

In [ ]:
clfRF = RandomForestClassifier(n_estimators=500, max_depth=150, min_samples_split=2, min_samples_leaf=1, max_features='sqrt')
clfRF.fit(X_train_ly16_rf, y_train_ly16)

RandomForestClassifier(max_depth=150, max_features='sqrt', n_estimators=500)

In [ ]:
y_predicted = clfRF.predict(X_test_ly16_rf)

In [ ]:
accuracy_score(y_test_ly16, y_predicted)

0.8861734181452963

In [ ]:
precision_score(y_test_ly16, y_predicted)

0.8812877263581489

In [ ]:
recall_score(y_test_ly16, y_predicted)

0.8908474576271187

In [ ]:
f1_score(y_test_ly16, y_predicted)

0.8860418071476738

## ACE2

In [11]:
train_ace2 = pd.read_csv("/content/gdrive/MyDrive/BI_project/ACE2_train_data.csv")
test_ace2 = pd.read_csv("/content/gdrive/MyDrive/BI_project/ACE2_test_data.csv")

In [12]:
train_ace2_encoded = encoding_func(train_ace2)
test_ace2_encoded = encoding_func(test_ace2)

In [ ]:
X_train_ace2_rf = train_ace2_encoded.reshape(train_ace2_encoded.shape[0], (train_ace2_encoded.shape[1]*train_ace2_encoded.shape[2]))
X_test_ace2_rf = test_ace2_encoded.reshape(test_ace2_encoded.shape[0], (test_ace2_encoded.shape[1]*test_ace2_encoded.shape[2]))

In [13]:
y_train_ace2 = train_ace2['Label']
y_test_ace2 = test_ace2['Label']

In [ ]:
clfRF_ace2 = RandomForestClassifier(n_estimators=500, max_depth=150, min_samples_split=2, min_samples_leaf=1, max_features='sqrt')
clfRF_ace2.fit(X_train_ace2_rf, y_train_ace2)

RandomForestClassifier(max_depth=150, max_features='sqrt', n_estimators=500)

In [ ]:
y_predicted_ace2 = clfRF_ace2.predict(X_test_ace2_rf)

In [ ]:
accuracy_score(y_test_ace2, y_predicted_ace2)

0.9259455872594559

In [ ]:
precision_score(y_test_ace2, y_predicted_ace2)

0.9387652744847711

In [ ]:
recall_score(y_test_ace2, y_predicted_ace2)

0.9112596264494999

In [ ]:
f1_score(y_test_ace2, y_predicted_ace2)

0.9248079773615415

# LSTM

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [18]:
class MyDataset(Dataset):
    def __init__(self, features, labels):
        self.labels = labels
        self.features = features

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        features = self.features[idx]
        labels = self.labels[idx]
        return features, labels

In [19]:
class LSTMPredictor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers):
        # hidden_size == number of neurons 
        super().__init__()
        #self.embeding = nn.Linear(20, 10)
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True, num_layers=num_layers, dropout=0.1)
        self.fc = nn.Linear(hidden_size, 1) # Predict only one value
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        #x = self.embeding(x)
        #h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size) #hidden state
        #c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size) #internal state
        out, (h,c) = self.lstm(x)
        #print(h.shape) 
        y = self.fc(h[self.num_layers - 1])
        y = self.sigmoid(y)
        return y

## Ly16

In [15]:
X_train_ly16_nn = torch.Tensor(train_ly16_encoded)
X_test_ly16_nn = torch.Tensor(test_ly16_encoded)

NameError: ignored

In [19]:
X_train_ly16_nn.shape

torch.Size([26881, 24, 20])

In [24]:
y_train_ly16_nn = torch.Tensor(y_train_ly16)
y_test_ly16_nn = torch.Tensor(y_test_ly16)

In [27]:
train_dataset_ly16 = MyDataset(X_train_ly16_nn, y_train_ly16_nn)
test_dataset_ly16 = MyDataset(X_test_ly16_nn, y_test_ly16_nn)

In [39]:
batch_size = 32
trainloader_ly16 = DataLoader(train_dataset_ly16, batch_size=batch_size, shuffle=True, num_workers=2)
testloader_ly16 = DataLoader(test_dataset_ly16, batch_size=batch_size, shuffle=False, num_workers=2)

In [34]:
lstm_ly16 =  LSTMPredictor(input_size = 20, hidden_size = 20, num_layers=3)

In [35]:
lstm_ly16.train()

num_epochs = 20
learning_rate = 0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(lstm_ly16.parameters(), lr=learning_rate)

loss_hist = []
for ep in range(num_epochs):
    hist_loss = 0
    for _, data in enumerate(trainloader_ly16, 0): # get bacth
        # parse batch
        features, labels = data
        # sets the gradients of all optimized tensors to zero.
        optimizer.zero_grad() 
        # get outputs
        outputs = lstm_ly16(features) 
        # calculate loss
        #print(outputs.shape, labels.unsqueeze(1).shape)
        loss = criterion(outputs, labels.unsqueeze(1))
        # calculate gradients
        loss.backward() 
        # performs a single optimization step (parameter update).
        optimizer.step()
        hist_loss += loss.item()
    loss_hist.append(hist_loss /len(trainloader_ly16))
    print(f"Epoch={ep} loss={loss_hist[ep]:.4f}")

Epoch=0 loss=0.5670
Epoch=1 loss=0.4498
Epoch=2 loss=0.4129
Epoch=3 loss=0.3867
Epoch=4 loss=0.3609
Epoch=5 loss=0.3515
Epoch=6 loss=0.3418
Epoch=7 loss=0.3380
Epoch=8 loss=0.3319
Epoch=9 loss=0.3288
Epoch=10 loss=0.3248
Epoch=11 loss=0.3229
Epoch=12 loss=0.3197
Epoch=13 loss=0.3164
Epoch=14 loss=0.3157
Epoch=15 loss=0.3152
Epoch=16 loss=0.3114
Epoch=17 loss=0.3101
Epoch=18 loss=0.3104
Epoch=19 loss=0.3120


In [40]:
accuracy = []
precision = []
recall = []
f1 = []
lstm_ly16.eval()

with torch.no_grad(): # New variables in this block will be created with require_grad = False
  for data in testloader_ly16:
    features, labels = data 
    outputs = lstm_ly16(features)
    outputs = outputs.squeeze().round()
    accuracy.append(accuracy_score(labels, outputs))
    precision.append(precision_score(labels, outputs))
    recall.append(recall_score(labels, outputs))
    f1.append(f1_score(labels, outputs))


In [41]:
sum(accuracy) / len(accuracy)

0.8876329787234043

In [42]:
sum(precision) / len(precision)

0.8642711769041731

In [43]:
sum(recall) / len(recall)

0.9168147888206997

In [44]:
sum(f1) / len(f1)

0.8858114646936682

## ACE2

In [16]:
X_train_ace2_nn = torch.Tensor(train_ace2_encoded)
X_test_ace2_nn = torch.Tensor(test_ace2_encoded)

In [17]:
y_train_ace2_nn = torch.Tensor(y_train_ace2)
y_test_ace2_nn = torch.Tensor(y_test_ace2)

In [20]:
train_dataset_ace2 = MyDataset(X_train_ace2_nn, y_train_ace2_nn)
test_dataset_ace2 = MyDataset(X_test_ace2_nn, y_test_ace2_nn)

batch_size = 32
trainloader_ace2 = DataLoader(train_dataset_ace2, batch_size=batch_size, shuffle=True, num_workers=2)
testloader_ace2 = DataLoader(test_dataset_ace2, batch_size=batch_size, shuffle=False, num_workers=2)

In [21]:
lstm_ace2 =  LSTMPredictor(input_size = 20, hidden_size = 20, num_layers=3)

In [22]:
lstm_ace2.train()

num_epochs = 20
learning_rate = 0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(lstm_ace2.parameters(), lr=learning_rate)

loss_hist = []
for ep in range(num_epochs):
    hist_loss = 0
    for _, data in enumerate(trainloader_ace2, 0): # get bacth
        # parse batch
        features, labels = data
        # sets the gradients of all optimized tensors to zero.
        optimizer.zero_grad() 
        # get outputs
        outputs = lstm_ace2(features) 
        # calculate loss
        #print(outputs.shape, labels.unsqueeze(1).shape)
        loss = criterion(outputs, labels.unsqueeze(1))
        # calculate gradients
        loss.backward() 
        # performs a single optimization step (parameter update).
        optimizer.step()
        hist_loss += loss.item()
    loss_hist.append(hist_loss /len(trainloader_ace2))
    print(f"Epoch={ep} loss={loss_hist[ep]:.4f}")

Epoch=0 loss=0.2771
Epoch=1 loss=0.2188
Epoch=2 loss=0.2119
Epoch=3 loss=0.2078
Epoch=4 loss=0.2049
Epoch=5 loss=0.2028
Epoch=6 loss=0.2012
Epoch=7 loss=0.1999


KeyboardInterrupt: ignored

In [23]:
accuracy = []
precision = []
recall = []
f1 = []
lstm_ace2.eval()

with torch.no_grad(): # New variables in this block will be created with require_grad = False
  for data in testloader_ace2:
    features, labels = data 
    outputs = lstm_ace2(features)
    outputs = outputs.squeeze().round()
    accuracy.append(accuracy_score(labels, outputs))
    precision.append(precision_score(labels, outputs))
    recall.append(recall_score(labels, outputs))
    f1.append(f1_score(labels, outputs))

In [24]:
sum(accuracy) / len(accuracy)

0.9321649246012305

In [25]:
sum(precision) / len(precision)

0.9472192882229037

In [27]:
sum(recall) / len(recall)

0.9147749369319526

In [26]:
sum(f1) / len(f1)

0.9286347626157505